## TBX11K Training with Augmented Images and CBAM 

### At this stage, we are going to implement the Convolutional Block Attention Model to our YOLOv8 small model. This is to improve the object detection capabilities of the model for X-Ray scans classified as Latent TB.

In [ ]:
# Install YOLO, Headless OpenCV (for server), and Data Tools
!pip install ultralytics opencv-python-headless albumentations gdown

## Downloading augmented dataset for training

In [ ]:
import gdown
import os

# Download dataset 
url = f'https://drive.google.com/drive/u/1/folders/{file_id}'
output = 'augmented_dataset.zip'

# CONSTRUCT URL: Use the "uc" (export) endpoint for standard files
url = f'https://drive.google.com/uc?id={file_id}'

# 3. DOWNLOAD
if not os.path.exists(output):
    print(f"Downloading {file_id}...")
    gdown.download(url, output, quiet=False, fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1FVoKWGlbjbKfAiQT1Ad33dK2GPmRXEcb
From (redirected): https://drive.google.com/uc?id=1FVoKWGlbjbKfAiQT1Ad33dK2GPmRXEcb&confirm=t&uuid=01c46a5d-bc6f-450f-b531-176d9f3fcf40
To: /augmented_dataset.zip
100%|██████████| 2.55G/2.55G [00:44<00:00, 56.8MB/s]


In [7]:
import zipfile
import os

zip_path = 'augmented_dataset.zip'
extract_path = "/workspace/dataset"

# Create folder if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

print(f"Extracting {zip_path}...")

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Success. Files extracted to:", extract_path)

Extracting augmented_dataset.zip...
Success. Files extracted to: /workspace/dataset


## Implementing a Convolutional Block Attention Model (CBAM) to YOLOv8 small
#### First, we define attention modules 

In [32]:
import torch
import torch.nn as nn
import contextlib
from copy import deepcopy
from ultralytics import YOLO
from ultralytics.nn import tasks
from ultralytics.nn.tasks import DetectionModel, make_divisible
from ultralytics.nn.modules import Conv, C2f, SPPF, Concat, Bottleneck

# =============================================================================
# DEFINE CBAM & C2f_CBAM
# =============================================================================
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc = nn.Sequential(
            nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        return self.sigmoid(avg_out + max_out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        padding = 3 if kernel_size == 7 else 1
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x_cat = torch.cat([avg_out, max_out], dim=1)
        return self.sigmoid(self.conv1(x_cat))

class CBAM(nn.Module):
    def __init__(self, c1, kernel_size=7):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention(c1)
        self.spatial_attention = SpatialAttention(kernel_size)

    def forward(self, x):
        x = x * self.channel_attention(x)
        x = x * self.spatial_attention(x)
        return x

class C2f_CBAM(nn.Module):
    # Standard C2f signature + CBAM at the end
    def __init__(self, c1, c2, n=1, shortcut=False, g=1, e=0.5):
        super().__init__()
        self.c = int(c2 * e) 
        self.cv1 = Conv(c1, 2 * self.c, 1, 1)
        self.cv2 = Conv((2 + n) * self.c, c2, 1) 
        self.m = nn.ModuleList(
            Bottleneck(self.c, self.c, shortcut, g, k=((3, 3), (3, 3)), e=1.0) 
            for _ in range(n)
        )
        self.cbam = CBAM(c2) 

    def forward(self, x):
        y = list(self.cv1(x).split((self.c, self.c), 1))
        y.extend(m(y[-1]) for m in self.m)
        out = self.cv2(torch.cat(y, 1))
        return self.cbam(out)

# Inject into modules namespace so eval() works
sys.modules['ultralytics.nn.modules'].C2f_CBAM = C2f_CBAM

## Register modules and build model

In [36]:
def custom_parse_model(d, ch, verbose=True):  # model_dict, input_channels(3)
    import ast

    # --- FIX: Handle integer input for 'ch' ---
    if isinstance(ch, int):
        ch = [ch]
 
    nc = d['nc']
    
    _ = d.get('width_multiple', 1.0) 
    _ = d.get('depth_multiple', 1.0)
    
    layers, save, c2 = [], [], ch[-1]  # layers, savelist, ch out
    
    for i, (f, n, m, args) in enumerate(d['backbone'] + d['head']):
        m = getattr(torch.nn, m[3:]) if 'nn.' in m else globals()[m] if m in globals() else getattr(tasks, m, None)
        
        for j, a in enumerate(args):
            if isinstance(a, str):
                try:
                    args[j] = locals()[a] if a in locals() else ast.literal_eval(a)
                except:
                    pass

        n = n_ = max(round(n * d.get('depth_multiple', 1.0)), 1) if n > 1 else n
        
        # Add C2f_CBAM to the whitelist so it calculates c1/c2 correctly
        if m in (Conv, C2f, C2f_CBAM, SPPF, Bottleneck): 
            c1, c2 = ch[f], args[0]
            if c2 != nc: 
                c2 = make_divisible(c2 * d.get('width_multiple', 1.0), 8)

            args = [c1, c2, *args[1:]]
            if m in (C2f, C2f_CBAM):
                args.insert(2, n)
                n = 1
        
        elif m is nn.BatchNorm2d:
            args = [ch[f]]
        elif m is Concat:
            c2 = sum(ch[x] for x in f)
        elif m is nn.Upsample:
            c2 = ch[f]
        elif m is getattr(tasks, 'Detect', None):
            args.append([ch[x] for x in f])
        else:
            c2 = ch[f]

        m_ = nn.Sequential(*(m(*args) for _ in range(n))) if n > 1 else m(*args)
        
        t = str(m)[8:-2].replace('__main__.', '')
        m_.i, m_.f, m_.type = i, f, t
        save.extend(x % i for x in ([f] if isinstance(f, int) else f) if x != -1)
        layers.append(m_)
        if i == 0:
            ch = []
        ch.append(c2)
        
    return nn.Sequential(*layers), sorted(save)

# Apply the patch
tasks.parse_model = custom_parse_model
print("Patched parse_model (v2) to recognize C2f_CBAM")

Patched parse_model (v2) to recognize C2f_CBAM


In [37]:
## BUILD MODEL
def get_cbam_model(nc=2):
    return DetectionModel(cfg={
        'nc': nc,
        'depth_multiple': 0.33,
        'width_multiple': 0.50,
        'backbone': [
            [-1, 1, 'Conv', [64, 3, 2]],
            [-1, 1, 'Conv', [128, 3, 2]],
            [-1, 3, 'C2f', [128, True]],
            [-1, 1, 'Conv', [256, 3, 2]],
            [-1, 6, 'C2f_CBAM', [256, True]], # Standard YOLO args: [c2, shortcut]
            [-1, 1, 'Conv', [512, 3, 2]],
            [-1, 6, 'C2f_CBAM', [512, True]], 
            [-1, 1, 'Conv', [1024, 3, 2]],
            [-1, 3, 'C2f_CBAM', [1024, True]],
            [-1, 1, 'SPPF', [1024, 5]]
        ],
        'head': [
            [-1, 1, 'nn.Upsample', [None, 2, 'nearest']],
            [[-1, 6], 1, 'Concat', [1]],
            [-1, 3, 'C2f', [512]], 

            [-1, 1, 'nn.Upsample', [None, 2, 'nearest']],
            [[-1, 4], 1, 'Concat', [1]],
            [-1, 3, 'C2f', [256]], 

            [-1, 1, 'Conv', [256, 3, 2]],
            [[-1, 12], 1, 'Concat', [1]],
            [-1, 3, 'C2f_CBAM', [512, False]], # Head shortcut usually False

            [-1, 1, 'Conv', [512, 3, 2]],
            [[-1, 9], 1, 'Concat', [1]],
            [-1, 3, 'C2f_CBAM', [1024, False]], 

            [[15, 18, 21], 1, 'Detect', [nc]] 
        ]
    }, ch=3, nc=nc)

## Creating the YAML file needed for YOLOv8 training

In [3]:
%%writefile /workspace/dataset/yolo_official/data.yaml
path: /workspace/dataset/yolo_official
train: images/train
val: images/val
nc: 2
names:
  0: ActiveTuberculosis
  1: LatentTuberculosis

Writing /workspace/dataset/yolo_official/data.yaml


## Training the YOLOv8 + CBAM model 
#### Change the image size to 640x640 when training for second time.

In [ ]:
if __name__ == '__main__':
    print("Building Custom CBAM Model...")
    custom_model = get_cbam_model(nc=2)

    print("Loading YOLOv8s weights (Transfer Learning)...")
    # 1. Load the standard pre-trained weights
    temp_model = YOLO('yolov8s.pt')
    pretrained_dict = temp_model.model.state_dict()
    
    #  Get the state of custom model
    custom_dict = custom_model.state_dict()
    
    # FILTER: Only keep weights that have the same shape
    # This automatically excludes the final 'Detect' layer (80 classes vs 2 classes)
    # and ignores the missing CBAM layers.
    pretrained_dict = {k: v for k, v in pretrained_dict.items() 
                       if k in custom_dict and v.shape == custom_dict[k].shape}
    
    # Update and Load
    custom_dict.update(pretrained_dict)
    custom_model.load_state_dict(custom_dict)
    print(f"Loaded {len(pretrained_dict)}/{len(custom_dict)} layers from pretrained weights.")
    
    # Create Runner
    runner = YOLO('yolov8s.pt')
    runner.model = custom_model
    
    print(" Starting Training...")
    runner.train(
        data='/workspace/dataset/yolo_official/data.yaml', 
        epochs=100,
        imgsz=512,
        batch=32,      
        patience=20,
        name='cbam_final_run',
        project='/workspace/runs',
        device=0,
        mosaic=1.0,
        hsv_h=0.0,
        hsv_s=0.0,
    )
    print("Training Complete!")

Building Custom CBAM Model...
Model summary: 169 layers, 11,220,832 parameters, 11,220,816 gradients, 28.8 GFLOPs

Loading YOLOv8s weights (Transfer Learning)...
Loaded 349/370 layers from pretrained weights.
 Starting Training...
Ultralytics 8.3.240 🚀 Python-3.11.10 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24081MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/workspace/dataset/yolo_official/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, ma

/opt/conda/lib/python3.11/site-packages/torch/autograd/graph.py:825: UserWarning: adaptive_max_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Context.cpp:91.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      1/100      4.66G       2.21      6.279       2.15         19        512: 100% ━━━━━━━━━━━━ 235/235 6.6it/s 35.5s<0.1s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 33/33 8.3it/s 4.0s0.1ss
                   all       2088        309     0.0167      0.333      0.024    0.00962

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      2/100      5.23G      2.071      3.087      1.808         12        512: 0% ──────────── 1/235 2.1it/s 0.3s<1:54

/opt/conda/lib/python3.11/site-packages/torch/autograd/graph.py:825: UserWarning: adaptive_max_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Context.cpp:91.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      2/100      5.24G      2.003      3.218      1.822         13        512: 100% ━━━━━━━━━━━━ 235/235 7.9it/s 29.6s<0.1s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 33/33 7.3it/s 4.5s0.2s
                   all       2088        309      0.153      0.217     0.0944       0.03

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      3/100      5.24G      2.199      5.821      1.916          1        512: 0% ──────────── 1/235 2.3it/s 0.3s<1:41

...

     93/100      5.24G     0.9118     0.5735      1.133          9        512: 100% ━━━━━━━━━━━━ 235/235 10.4it/s 22.7s0.1ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 33/33 13.7it/s 2.4s.1ss
                   all       2088        309      0.467      0.343      0.386      0.178

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     94/100      5.24G      0.813     0.6187      1.123         11        512: 0% ──────────── 1/235 1.4it/s 0.2s<2:46

/opt/conda/lib/python3.11/site-packages/torch/autograd/graph.py:825: UserWarning: adaptive_max_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Context.cpp:91.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


     94/100      5.24G     0.8972     0.5481      1.126         11        512: 100% ━━━━━━━━━━━━ 235/235 10.3it/s 22.7s0.2ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 33/33 13.7it/s 2.4s.1ss
                   all       2088        309       0.48      0.339      0.387      0.178

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     95/100      5.24G     0.8768     0.4381      1.092          8        512: 0% ──────────── 0/235  0.1s

/opt/conda/lib/python3.11/site-packages/torch/autograd/graph.py:825: UserWarning: adaptive_max_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Context.cpp:91.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


     95/100      5.24G     0.8957     0.5627      1.128          8        512: 100% ━━━━━━━━━━━━ 235/235 10.5it/s 22.4s<0.1s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 33/33 13.7it/s 2.4s.1ss
                   all       2088        309      0.482      0.335      0.385      0.181
EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 75, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

95 epochs completed in 0.772 hours.
Optimizer stripped from /workspace/runs/cbam_final_run/weights/last.pt, 22.7MB
Optimizer stripped from /workspace/runs/cbam_final_run/weights/best.pt, 22.7MB

Validating /workspace/runs/cbam_final_run/weights/best.pt...
Ultralytics 8.3.240 🚀 Python-3.11.10 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24081MiB)
Model summary (fused)